In [215]:
import csv
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time
from pprint import pprint
from datetime import datetime
import pytz
import shutil

In [216]:
version='23_24_gw1'
league_code = "551115"

In [217]:
start=time.perf_counter()

In [218]:
#Request league standings and return all league data 
def get_json_from_url(league_code):
    league_url="https://fantasy.premierleague.com/api/leagues-classic/"+league_code+"/standings/?page_new_entries=1&page_standings=1&phase=1"
    get=requests.get(league_url)
    league_data=json.loads(get.text)
    league_name = league_data['league'].get('name')
    print("Data fetched, league name: ", league_name)
    return league_data, league_name

#Use league data to return a dict of names & entry ids
def make_ids(league_data):
    member_ids={}
    for member in league_data['standings']['results']:
        name=member.get('player_name')
        mid=member.get('entry')
        member_ids[name]=mid
    return member_ids

#Use first value in member_ids to find the max gameweek that data is available for
def find_max_gw(xid): #xid=member entry id
    gw=1
    while gw < 100:
        try:
            get_mgw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_mgw=json.loads(get_mgw.text)
            list(data_mgw['entry_history'].keys())
            gw+=1
        except KeyError as err:
            print("Max gameweek data available for is: ",gw)
            return (gw)
            break

In [219]:
#(Doesn't work until GW1)
league_data, league_name = get_json_from_url(league_code)
member_ids=make_ids(league_data)
mgw=find_max_gw(list(member_ids.values())[0])

TypeError: string indices must be integers, not 'str'

In [ ]:
#Get data1
def get_first_member_data(member_ids):
    xid=list(member_ids.values())[0]
    gw=1
    get_first=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
    data_first=json.loads(get_first.text)
    return data_first

#Member picks
def dataframe_from_ids(member_ids,cols):
    p=[]

     #member data

    for xid in member_ids.values():
        for gw in range(1,mgw):    
            get_gw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_gw=json.loads(get_gw.text)

            for x in range(0,15):
                #picks
                row=data_gw["picks"][x]

                name=list(member_ids.keys())[list(member_ids.values()).index(xid)]
                selected_row = [name,xid,gw,x+1]

                for i in cols:
                    selected_row.append(row.get(i))

                p.append(selected_row)

                print(xid,name,"GW "+str(gw),"Pick "+str(x+1))
    print("Finished")
    pcols=['member','memberid','gw','pick']
    pcols.extend(cols)
    picksdf = pd.DataFrame(p, columns=pcols)
    return picksdf
data_for_columns=get_first_member_data(member_ids)
cols=list(data_for_columns["picks"][0].keys())
picksdf=dataframe_from_ids(member_ids,cols)

2654380 Joe Langwith GW 1 Pick 1
2654380 Joe Langwith GW 1 Pick 2
2654380 Joe Langwith GW 1 Pick 3
2654380 Joe Langwith GW 1 Pick 4
2654380 Joe Langwith GW 1 Pick 5
2654380 Joe Langwith GW 1 Pick 6
2654380 Joe Langwith GW 1 Pick 7
2654380 Joe Langwith GW 1 Pick 8
2654380 Joe Langwith GW 1 Pick 9
2654380 Joe Langwith GW 1 Pick 10
2654380 Joe Langwith GW 1 Pick 11
2654380 Joe Langwith GW 1 Pick 12
2654380 Joe Langwith GW 1 Pick 13
2654380 Joe Langwith GW 1 Pick 14
2654380 Joe Langwith GW 1 Pick 15
5435760 Will Hossack GW 1 Pick 1
5435760 Will Hossack GW 1 Pick 2
5435760 Will Hossack GW 1 Pick 3
5435760 Will Hossack GW 1 Pick 4
5435760 Will Hossack GW 1 Pick 5
5435760 Will Hossack GW 1 Pick 6
5435760 Will Hossack GW 1 Pick 7
5435760 Will Hossack GW 1 Pick 8
5435760 Will Hossack GW 1 Pick 9
5435760 Will Hossack GW 1 Pick 10
5435760 Will Hossack GW 1 Pick 11
5435760 Will Hossack GW 1 Pick 12
5435760 Will Hossack GW 1 Pick 13
5435760 Will Hossack GW 1 Pick 14
5435760 Will Hossack GW 1 Pick 1

In [ ]:
#Insert into PSQL
from sqlalchemy import create_engine, text
from datetime import datetime
import psycopg2 # python -> psql connection
import psycopg2.extras

#Import secret settings
import CJDH_local_settings
dbname = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['dbname']
user = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['user']
password = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['password']
host = "localhost"
port = 5432

#Create connection to psql via psycopg2 & sqlalchemy
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
engine.autocommit = True  # read documentation understanding when to Use & NOT use # changes will be automatically committed to the database after each statement is executed.

In [ ]:
picksdf.columns
picks = picksdf[['member', 'memberid', 'gw', 'pick', 'element', 'position', 'multiplier',
       'is_captain', 'is_vice_captain']]

In [ ]:
#Create staging table for player data 
def create_staging_table_picks(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS picks CASCADE;
        CREATE UNLOGGED TABLE picks (
            member          TEXT,
            memberid        NUMERIC,
            gw              NUMERIC,
            pick            NUMERIC,
            element         NUMERIC,
            position        NUMERIC,
            multiplier      NUMERIC,
            is_captain      BOOLEAN,
            is_vice_captain BOOLEAN
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_picks(cursor)
picks.to_sql('picks', con=engine, if_exists='replace',index=False)

255

In [ ]:
query = """
        SELECT *
        FROM player_details
        LEFT JOIN player
            ON player_details.element = player.id
        ;
        """
with engine.connect() as conn:
    playerdf = pd.read_sql_query(query, conn)
playerdf.head()

,season,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,...,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,points_per_game_rank,selected_rank,starts_per_90,clean_sheets_per_90,now_datetime_utc,current_gw,next_gw
0,20232024,1,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0.0,0.00,0.0,461,198,0.0,0.0,2023-08-18 13:49:23.059834+00:00,1,2
1,20232024,2,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0.0,0.00,0.0,289,254,0.0,0.0,2023-08-18 13:49:23.059834+00:00,1,2
2,20232024,3,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0.0,0.00,0.0,419,452,0.0,0.0,2023-08-18 13:49:23.059834+00:00,1,2
3,20232024,4,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,...,0.0,0.00,0.0,641,417,0.0,0.0,2023-08-18 13:49:23.059834+00:00,1,2
4,20232024,5,2,16,1,True,2023-08-12T12:00:00Z,2,1,1,...,0.0,0.45,0.0,242,9,0.0,0.0,2023-08-18 13:49:23.059834+00:00,1,2


In [ ]:
me = 'Christopher Harris'
myid = set(picksdf[picksdf['member']==me]['memberid'])
mypicksdf = picksdf[picksdf['member']==me]
mypicks = set(mypicksdf['element'])
mypicks

#league_standings
l = pd.DataFrame(league_data['standings']['results'])
mytotal = l[l['entry'].isin(myid)]['total'].values
#get ids for members above me or less than 100 points below me
ids_close_to_me = set(l[l['total']<=int(mytotal)+100]['entry'][~l['entry'].isin(myid)])

In [ ]:
#Get ep_next * by ownership rate in mini-league
latestpicks = picksdf[(picksdf['gw']==max(picksdf['gw']))&(picksdf['memberid'].isin(ids_close_to_me))]
max_owns = len(set(latestpicks['memberid']))

ownership = pd.pivot_table(latestpicks, index=['element'],values='multiplier',aggfunc=np.sum).reset_index()
ownership['own_rate'] = ownership['multiplier']/max_owns

ownership.sort_values(by='own_rate', ascending=False)

playerdf = pd.merge(playerdf, ownership, how='outer',on='element')
filtered = playerdf['element'].isin(mypicks)
gainpicks = playerdf[filtered].merge(picksdf[picksdf['member']==me],how='left',on='element',suffixes=('','_myteam'))
gainpicks['expected_gain'] = gainpicks['ep_next'].astype(float)*(gainpicks['multiplier_myteam']-gainpicks['own_rate'])
gainpicks[['web_name','element','element_type','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)

,web_name,element,element_type,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain
9,B.Fernandes,373,3,5.0,1,0.3125,3.4,2.33750
3,Watkins,60,4,3.0,1,0.1875,2.6,2.11250
11,Onana,597,1,7.0,1,0.4375,3.7,2.08125
1,Martinelli,12,3,6.0,1,0.3750,3.3,2.06250
7,Chilwell,195,2,4.0,1,0.2500,2.6,1.95000
10,Rashford,396,3,8.0,1,0.5000,3.5,1.75000
8,Haaland,355,4,26.0,2,1.6250,4.5,1.68750
0,Gabriel,5,2,9.0,1,0.5625,2.8,1.22500
4,Estupiñan,131,2,10.0,1,0.6250,2.4,0.90000
2,Saka,19,3,12.0,1,0.7500,3.4,0.85000


In [ ]:
print("Expected Total Gain on Division Average")
gainpicks['expected_gain'].sum()

Expected Total Gain on Division Average


17.5

In [ ]:
#players not in my picks
losspicks = playerdf[~filtered].copy().dropna(subset='multiplier')
losspicks['expected_loss'] = losspicks['ep_next'].astype(float)*(0-losspicks['own_rate'])
print("Expected Total Loss on Division Average: ")
print(losspicks['expected_loss'].sum())
losspicks[['web_name','element','multiplier','own_rate','ep_next','expected_loss']].sort_values(by='expected_loss',ascending=True).head()

Expected Total Loss on Division Average: 
-13.6375


,web_name,element,multiplier,own_rate,ep_next,expected_loss
347,Salah,308,5.0,0.3125,4.5,-1.40625
329,Alexander-Arnold,290,4.0,0.2500,4.5,-1.12500
143,João Pedro,135,8.0,0.5000,1.9,-0.95000
333,Diogo J.,294,3.0,0.1875,3.3,-0.61875
452,Shaw,398,3.0,0.1875,3.1,-0.58125


In [ ]:
print("Net Gain")
print(round(gainpicks['expected_gain'].sum() + losspicks['expected_loss'].sum(),2))

Net Gain
3.86


In [ ]:
teamgains = gainpicks[['web_name','element','element_type','now_cost','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)
teamgains['expected_loss_on_transfer'] = teamgains['ep_next'].astype(float)*(teamgains['own_rate'])

allgains = playerdf.copy()
allgains['expected_gain'] = allgains['ep_next'].astype(float)*(1-allgains['own_rate'])
allgains[['web_name','element','element_type','now_cost','ep_next','own_rate','expected_gain']].sort_values(by='expected_gain',ascending=False).dropna().head(20)

def top_suggested_transfer(row, df):
    #get element, element_type & gain
    myteam_type = row['element_type']
    # myteam_gain = row['expected_gain']
    myteam_loss = row['expected_loss_on_transfer']
    myteam_cost = row['now_cost']
    myteam_element = row['element']
    #get playerdf element, element_type & gain
    filtered = (df['element_type']==myteam_type) & (df['expected_gain']>myteam_loss) & (df['now_cost']<=myteam_cost) & (df['element']!=myteam_element)
    best_gain = df[filtered].sort_values(by='expected_gain', ascending=False).head(1)
    best_gain['own_rate'] = round(best_gain['own_rate'],3)
    best_gain['expected_gain'] = round(best_gain['expected_gain'],2)
    best_gain['diff'] = round(df['expected_gain']-row['expected_loss_on_transfer'],2)
    return best_gain[['web_name','own_rate','ep_next','expected_gain','diff']].values[0]

teamgains['affordable_suggested_transfer'] = teamgains.apply(lambda x: top_suggested_transfer(x, allgains),axis=1)
# Extracting the second element from the arrays in the "affordable_suggested_transfer" column
teamgains['suggested_transfer_gain'] = teamgains['affordable_suggested_transfer'].apply(lambda x: x[-1])

# Sorting the DataFrame based on the second element in the arrays
teamgains = teamgains.sort_values(by='suggested_transfer_gain', ascending=False)
teamgains


,web_name,element,element_type,now_cost,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain,expected_loss_on_transfer,affordable_suggested_transfer,suggested_transfer_gain
7,Chilwell,195,2,56,4.0,1,0.2500,2.6,1.95000,0.65000,"[Rúben, 0.0, 3.1, 3.1, 2.45]",2.45
3,Watkins,60,4,80,3.0,1,0.1875,2.6,2.11250,0.48750,"[Darwin, 0.062, 2.9, 2.72, 2.23]",2.23
9,B.Fernandes,373,3,85,5.0,1,0.3125,3.4,2.33750,1.06250,"[Ødegaard, 0.062, 3.4, 3.19, 2.12]",2.12
1,Martinelli,12,3,80,6.0,1,0.3750,3.3,2.06250,1.23750,"[Havertz, 0.062, 3.1, 2.91, 1.67]",1.67
11,Onana,597,1,50,7.0,1,0.4375,3.7,2.08125,1.61875,"[Ramsdale, 0.125, 3.7, 3.24, 1.62]",1.62
10,Rashford,396,3,90,8.0,1,0.5000,3.5,1.75000,1.75000,"[Ødegaard, 0.062, 3.4, 3.19, 1.44]",1.44
4,Estupiñan,131,2,51,10.0,1,0.6250,2.4,0.90000,1.50000,"[Walker, 0.0, 2.8, 2.8, 1.3]",1.30
0,Gabriel,5,2,50,9.0,1,0.5625,2.8,1.22500,1.57500,"[Walker, 0.0, 2.8, 2.8, 1.22]",1.22
6,Beyer,160,2,40,0.0,0,0.0000,0.0,0.00000,0.00000,"[Chambers, 0.0, 1.0, 1.0, 1.0]",1.00
12,Turner,28,1,40,0.0,0,0.0000,1.5,0.00000,0.00000,"[Areola, 0.0, 1.0, 1.0, 1.0]",1.00


Gameweek Level Analysis

In [ ]:
#Optimum gameweek

def formationmax(pickpoints,gw,m):
    #list of formations possible
    formations_possible=[[1,3,5,2],[1,3,4,3],[1,4,5,1],[1,4,4,2],[1,4,3,3],[1,5,4,1],[1,5,3,2],[1,5,2,3]]
    formation_names=['352','343','451','442','433','541','532','523']
    
    bestformation=[]
    best_of_each_formation=[]
    
    for count,formation in enumerate(formations_possible):
        typescore=[]
        bestformation=[]
        for p_type in range(1,5):
            typenum=formations_possible[count][p_type-1]
            membergwtype=pickpoints[['member','gw','pick','element_type','element','web_name','element_type','total_points_gw']][(pickpoints['gw']==gw) & (pickpoints['member']==m) & (pickpoints['element_type']==p_type)].sort_values(by='total_points_gw',ascending=False)
            best_in_type=membergwtype[['element_type','element','web_name','total_points_gw']][:typenum]
            pick=list(best_in_type['web_name'].values)
            points=list(best_in_type['total_points_gw'].values)
            best_from_type=list(zip(pick,points))
            bestformation.extend(best_from_type)
            #for this player type in the form [id,points], sum points:
            s = [i[1] for i in best_from_type]     
            typescore.extend([sum(s)])             
        formationscore=sum(typescore)        
        captain=sorted(bestformation, key = lambda f: f[1],reverse=True)[0]        
        totalscore=formationscore+captain[1]       
        best_of_each_formation.append([gw,m,formation_names[count],totalscore,captain,formationscore,bestformation])
        
    formtn_max=sorted(best_of_each_formation, key = lambda x: x[3])[-1] #get total score)
    return formtn_max

def get_optimumdf(alldata):

    fdf=[]
    pickpoints=alldata[['member','gw','pick','element_type','element','web_name','total_points_gw']]

    for m in list(member_ids.keys()):
        for gw in range(1,38+1):
            try:
                x=formationmax(pickpoints,gw,m) #function defined above
                fdf.extend([x])
                print(m,gw)
            except IndexError as err:
                # print(m,gw,err)
                continue

    optimumdf=pd.DataFrame(fdf,columns=['gw','member','formation_name','best_score','captain','formationscore','formation_lineup'])
    return optimumdf

optimumdf=get_optimumdf(alldata)

Joe Langwith 1
Will Hossack 1
Christopher Harris 1
Anthony Collier 1
R Birdy 1
Stephen Collier 1
Robert Goswell 1
Ollie Carroll 1
Callum Harling 1
Jack Carroll 1
Michael Irvine 1
Ryan Boyce 1
Daniel Sadler 1
Aaron Avison 1
Conner Harling 1
Ben Shankland 1
Tajwar Shelim 1


In [ ]:
#Merge gameweek data
gameweek=alldata[['member','gw','gw_total']].groupby(['member','gw'],as_index=False).sum()
gameweek=gameweek.merge(optimumdf[['member','gw','formation_name','best_score','captain','formationscore','formation_lineup']], how='outer', on=('member','gw'),suffixes=('_gwdata', '_optimum'))

In [ ]:
# #Member Gameweeks
dfcsv=gameweek
dfcsv.to_csv(r'data\DIVX_gameweek'+str(version)+'.csv')
dfcsv.to_csv(r'data\DIVX_gameweek_updated.csv')

#Member Picks
dfcsv=alldata
dfcsv.to_csv(r'data\DIVX_picks'+str(version)+'.csv')
dfcsv.to_csv(r'data\DIVX_picks_updated.csv')

In [ ]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#17/08/23: Finished in 3.59 minute(s)

Finished in 0.23 minute(s)
